# Pierre's modelization notebook

## Import of modules

In [39]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Analysis of features and target

#### Opening of the DataFrame

In [40]:
df = pd.read_csv('../../data/preprocessed_data.csv')
df.head()

,C concentration (weight%),Si concentration (weight%),Mn concentration (weight%),S concentration (weight%),P concentration (weight%),V concentration (weight%),O concentration (ppm/weight),Ti concentration (ppm/weight),N concentration (ppm/weight),Al concentration (ppm/weight),...,electrode positive,Type of weld_GMAA,Type of weld_GTAA,Type of weld_MMA,Type of weld_NGGMA,Type of weld_NGSAW,Type of weld_SA,Type of weld_SAA,Type of weld_ShMA,Type of weld_TSA
0,-1.794624,-0.241903,-1.474090,-0.150353,-0.062002,1.542981e-17,-1.143271e-16,0.0,0.0,8.694071e-17,...,True,False,False,True,False,False,False,False,False,False
1,-1.794624,-0.241903,-1.474090,-0.150353,-0.062002,1.542981e-17,-1.143271e-16,0.0,0.0,8.694071e-17,...,True,False,False,True,False,False,False,False,False,False
2,-1.794624,-0.157085,-0.467487,-0.234530,0.034196,1.542981e-17,-1.143271e-16,0.0,0.0,8.694071e-17,...,True,False,False,True,False,False,False,False,False,False
3,-1.794624,-0.157085,-0.467487,-0.234530,0.034196,1.542981e-17,-1.143271e-16,0.0,0.0,8.694071e-17,...,True,False,False,True,False,False,False,False,False,False
4,-1.474696,0.182188,0.592095,-0.234530,0.034196,1.542981e-17,-1.143271e-16,0.0,0.0,8.694071e-17,...,True,False,False,True,False,False,False,False,False,False


#### Deletion of the useless columns for the prediction and determination of the best target columns

We have various steel, yet the charpy temperature seems to have huge differences depending of the type.

In [41]:
# Drop the columns that are not needed
df = df.drop('Charpy temperature (deg C)',axis=1)

# Get the name of the target columns 
testNames = ['Yield strength (MPa)','Ultimate tensile strength (MPa)', 'Elongation (%)',
            'Reduction of Area (%)','Charpy impact toughness (J)']

# Get the name of the feature columns
featureNames = df.columns.difference(testNames)

### Information printing
This part is based on Clarisse's work.

This part will be usefull to decide how the target will be created with the columns referring to the welding test quality (Can be called "test columns").

We start with a Boolean columns creation : see for each row if the test is done.

In [42]:
# Initialize the dataframe that will contain the new columns
dfAugmented = df.copy()

# Initialize the list which will contain the name of the new columns
newColNames=['nb']

# Create a column that will tell how many test each observation has
# Initialize all the values to 0
dfAugmented['nb'] = 0

# Create a column, composed of boolean, with the suffix "_test" for each test column
# to see if the test is done or not (blank cells or not) and a column that will tell
# how many test each observation has
for c in testNames:
    n=c+'_test'
    dfAugmented[n] = np.where(dfAugmented[c].isna(),0,1)
    newColNames.append(n)
    dfAugmented['nb'] = dfAugmented['nb'] + dfAugmented[n]

In [43]:
# Display of test completion rate
for k in newColNames[1:]:
    dfInter = dfAugmented[k].value_counts().reset_index()
    dfInter['count'] = (dfInter['count']*100)/dfInter['count'].sum()
    print(k,'\nTest not done',int(dfInter['count'].to_list()[0]), '%\nTest done',int(dfInter['count'].to_list()[1]),'%\n')

# Display the number of tests performed on each observation
dfAugmented['nb'].value_counts()


Yield strength (MPa)_test 
Test not done 52 %
Test done 47 %

Ultimate tensile strength (MPa)_test 
Test not done 50 %
Test done 49 %

Elongation (%)_test 
Test not done 53 %
Test done 46 %

Reduction of Area (%)_test 
Test not done 52 %
Test done 47 %

Charpy impact toughness (J)_test 
Test not done 50 %
Test done 49 %



nb
1    650
4    510
5    134
0     82
3     52
2     35
Name: count, dtype: int64

Conclusion : 

Nearly half of the instances have only one test. All tests are done approximately 50% of the time. There are only 82 instances out of 1652 which do not have any tests.

### Preparation of the columns that are use to create the target 
Due to the fact that there is less than 5% of the dataset which has no test done. I choose to drop these instances.

In [46]:
# Drop the values where no test was done
dfAugmented = dfAugmented.drop(dfAugmented[dfAugmented['nb'] == 0].index)

# Verify if only 82 instances have been dropped
print(f"Number of dropped instances : {df.shape[0]-dfAugmented.shape[0]}")

Number of dropped instances : 82
